In [1]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from prophet import Prophet


In [2]:
df15 = pd.read_csv('/Users/federicogamberini/VS Code/when-to-buy-iphone/Dataset/iPhone15.csv')
df16 = pd.read_csv('/Users/federicogamberini/VS Code/when-to-buy-iphone/Dataset/iPhone16.csv')

In [3]:
# Filtra solo i dati comuni (fino all'ultimo gionro utile)
max_days = min(df15['Giorni_dal_lancio'].max(), df16['Giorni_dal_lancio'].max(), len(df16))

df15_md = df15[df15['Giorni_dal_lancio'] <= max_days].copy()
df16_md = df16[df16['Giorni_dal_lancio'] <= max_days].copy()
# Rinomina le colonne prezzo per evitare confusione
df15_md.rename(columns={'Prezzo': 'Prezzo_15'}, inplace=True)
df16_md.rename(columns={'Prezzo': 'Prezzo_16'}, inplace=True)
# Unione dei due dataframe per giorno dal lancio
df_merge = pd.merge(df16_md, df15_md[['Giorni_dal_lancio', 'Prezzo_15']], on='Giorni_dal_lancio')

In [4]:
# Prophet richiede colonne chiamate 'ds' (data) e 'y' (valore target)
df_merge['ds'] = pd.to_datetime(df_merge['Data'])   # Converte la colonna Data in datetime, necessaria per Prophet
df_merge['y'] = df_merge['Prezzo_16']               # Il prezzo che vogliamo prevedere (target)

# Definzione del regressore esterno: i prezzi dell’iPhone 15 nei giorni corrispondenti
df_merge['Prezzo_15'] = df_merge['Prezzo_15']       # Redondante, ma utile per chiarezza
df_prophet = df_merge[['ds', 'y', 'Prezzo_15']]     # Prophet userà solo queste colonne

model = Prophet()
# Aggiunta del regressore esterno (Prezzo iPhone 15 come indicatore del prezzo iPhone 16)
model.add_regressor('Prezzo_15')
# Allineamento del modello sui dati storici di iPhone 16 e i prezzi corrispondenti di iPhone 15
model.fit(df_prophet)

07:05:42 - cmdstanpy - INFO - Chain [1] start processing
07:05:42 - cmdstanpy - INFO - Chain [1] done processing


In [5]:
# Definizione il numero di giorni futuri da prevedere:
future_days_3m = 90    # circa 3 mesi
future_days_6m = 180   # circa 6 mesi -> usato

#L’ultima data disponibile nei dati di iPhone 16
last_date = df_prophet['ds'].max()                    # Ultima data nei dati
last_day = df_merge['Giorni_dal_lancio'].max()        # Ultimo giorno numerico (es: Giorno 280)

# Array di giorni futuri da predire, da (ultimo giorno + 1) fino a 6 mesi dopo
future_days = np.arange(last_day + 1, last_day + future_days_6m + 1)

# Per poter usare Prezzo_15 come regressore nei giorni futuri, bisonga costruire una lista corrispondente
df15_future = df15_md.set_index('Giorni_dal_lancio')  # si usa Giorni_dal_lancio come indice per accesso diretto

future_prezzi_15 = []  # Lista dove mettere i prezzi dell’iPhone 15 per ciascun giorno futuro

for day in future_days:
    if day in df15_future.index:
        # Se si ha un valore storico per quel giorno nei dati di iPhone 15, lo usa
        future_prezzi_15.append(df15_future.loc[day, 'Prezzo_15'])
    else:
        # Altrimenti si usa l’ultimo prezzo noto (ipotesi semplice per estendere la serie)
        future_prezzi_15.append(df15_future['Prezzo_15'].iloc[-1])

# Si generano delle date future corrispondenti a questi giorni
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=future_days_6m)

# Si crea il dataframe per la previsione: data e regressore esterno
future_df = pd.DataFrame({
    'ds': future_dates,            # Colonna con le date future
    'Prezzo_15': future_prezzi_15  # Valori di regressore esterno stimati (iPhone 15)
})

# Modello Prophet per fare la previsione sui dati futuri
forecast = model.predict(future_df)

# Estrazione dei risultati per i prossimi 3 e 6 mesi
forecast_3m = forecast.iloc[:future_days_3m]   # Prime 90 righe = 3 mesi
forecast_6m = forecast.iloc[:future_days_6m]   # Prime 180 righe = 6 mesi

# yhat è la previsione centrale, yhat_lower e yhat_upper l'intervallo di confidenza
print(forecast_3m[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])
print(forecast_6m[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

           ds        yhat  yhat_lower  yhat_upper
0  2025-08-21  761.282115  737.772103  784.115299
1  2025-08-22  756.830584  733.575063  779.628353
2  2025-08-23  755.841799  730.792228  780.035272
3  2025-08-24  758.529651  734.416522  782.919719
4  2025-08-25  759.529855  735.905681  782.995766
..        ...         ...         ...         ...
85 2025-11-14  748.071931  694.261025  803.457610
86 2025-11-15  747.083147  690.980326  805.959140
87 2025-11-16  749.770999  694.540982  806.378785
88 2025-11-17  750.771203  695.655528  809.220084
89 2025-11-18  754.623533  698.898294  812.228686

[90 rows x 4 columns]
            ds        yhat  yhat_lower  yhat_upper
0   2025-08-21  761.282115  737.772103  784.115299
1   2025-08-22  756.830584  733.575063  779.628353
2   2025-08-23  755.841799  730.792228  780.035272
3   2025-08-24  758.529651  734.416522  782.919719
4   2025-08-25  759.529855  735.905681  782.995766
..         ...         ...         ...         ...
175 2026-02-12  743.

In [6]:
from prophet.plot import plot_components_plotly
fig = plot_components_plotly(model, forecast)
fig.show()

In [7]:
iphone16_real = df_merge[['Giorni_dal_lancio', 'Prezzo_16']] # Dati reali iPhone 16
iphone16_pred = forecast_6m.copy() # Dati predetti iPhone 16
iphone16_pred['Giorni_dal_lancio'] = np.arange(last_day + 1, last_day + future_days_6m + 1) 
iphone15 = df15[['Giorni_dal_lancio', 'Prezzo']] # Dati iPhone 15


fig = go.Figure()
# iPhone 15 - linea di confronto (secondo piano)
day_plot = len(iphone16_real)+len(forecast_6m)
fig.add_trace(go.Scatter(x=iphone15['Giorni_dal_lancio'][:day_plot],y=iphone15['Prezzo'],name='iPhone 15 (storico)',
    line=dict(color='royalblue', width=2),opacity=0.8))
# iPhone 16 - storico
fig.add_trace(go.Scatter(x=iphone16_real['Giorni_dal_lancio'],y=iphone16_real['Prezzo_16'],
    name='iPhone 16 (storico)',line=dict(color='cyan', width=2)))
# iPhone 16 - previsione
fig.add_trace(go.Scatter(x=iphone16_pred['Giorni_dal_lancio'],y=iphone16_pred['yhat'],mode='lines',
    name='iPhone 16 (previsione)',line=dict(color='orange', width=2)))
# Intervallo di confidenza
fig.add_trace(go.Scatter(x=list(iphone16_pred['Giorni_dal_lancio']) + list(iphone16_pred['Giorni_dal_lancio'])[::-1],
    y=list(iphone16_pred['yhat_upper']) + list(iphone16_pred['yhat_lower'])[::-1],fill='toself',fillcolor='rgba(255,165,0,0.2)',
    line=dict(color='rgba(255,255,255,0)'),hoverinfo="skip",showlegend=True,name='Intervallo di confidenza'))
# Linea verticale: oggi (fine dati storici)
fig.add_shape(type='line',x0=last_day, y0=min(df_merge['Prezzo_16'].min(), iphone15['Prezzo'].min()),
    x1=last_day,y1=max(df_merge['Prezzo_16'].max(), iphone15['Prezzo'].max()),
    line=dict(color='gray', width=1, dash='dot'),name='Fine storico')

fig.update_layout(
    title='Prezzo iPhone 16 (storico e previsto) con confronto iPhone 15',
    xaxis_title='Giorni dal lancio', yaxis_title='Prezzo (€)', template='plotly_dark')

fig.show()